In [38]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
#### calculate miss values
def col_miss(train_df):
    col_missing_df = train_df.isnull().sum(axis=0).reset_index()
    col_missing_df.columns = ['col','missing_count']
    col_missing_df = col_missing_df.sort_values(by='missing_count')
    return col_missing_df

#### obtain cols of XX type
def obtain_x(train_df,xtype):
    dtype_df = train_df.dtypes.reset_index()
    dtype_df.columns = ['col','type']
    return dtype_df[dtype_df.type==xtype].col.values

def date_cols(train_df,float_col):
    float_date_col = []
    for col in float_col:
        if train_df[col].min() > 1e13:
            float_date_col.append(col)
    return float_date_col

def float_uniq(float_df,float_col):
    float_uniq_col = []
    for col in float_col:
        uniq = float_df[col].unique()
        if len(uniq) == 1:
            float_uniq_col.append(col)
    return float_uniq_col

def cal_corrcoef(float_df,y_train,float_col):
    corr_values = []
    for col in float_col:
        corr_values.append(abs(np.corrcoef(float_df[col].values,y_train)[0,1]))
    corr_df = pd.DataFrame({'col':float_col,'corr_value':corr_values})
    corr_df = corr_df.sort_values(by='corr_value',ascending=False)
    return corr_df

def build_model(x_train,y_train):
    reg_model = LinearRegression()
    reg_model.fit(x_train,y_train)
    return reg_model

In [39]:
train_df = pd.read_excel('train.xlsx')

In [50]:
# calculate the number of miss values
col_missing_df = col_miss(train_df)
# del cols of all nan
all_nan_columns = col_missing_df[col_missing_df.missing_count==499].col.values
train_df.drop(all_nan_columns,axis=1,inplace=True)
# obtain float cols
float64_col = obtain_x(train_df,'float64')
# del cols that miss number greater than 200
miss_float = train_df[float64_col].isnull().sum(axis=0).reset_index()
miss_float.columns = ['col','count']
miss_float_almost = miss_float[miss_float['count']>200].col.values
float64_col = float64_col.tolist()
float64_col = [col for col in float64_col if col not in miss_float_almost]
# del date cols
float64_date_col = date_cols(train_df,float64_col)
float64_col = [col for col in float64_col if col not in float64_date_col]

In [51]:
float_df = train_df[float64_col].copy()

In [52]:
median=float_df.median()

In [53]:
float_df.fillna(median,inplace=True)

In [54]:
# del cols which unique eq. 1
float64_uniq_col = float_uniq(float_df,float64_col)
float64_col = [col for col in float64_col if col not in float64_uniq_col]
# obtained corrcoef greater than 0.2
float64_col.remove('Y')
y_train = train_df.Y.values
corr_df = cal_corrcoef(float_df,y_train,float64_col)
corr02 = corr_df[corr_df.corr_value>=0.2]
corr02_col = corr02['col'].values.tolist()
x_train = float_df[corr02_col].values
test_df = pd.read_excel('testA.xlsx')
sub_test = test_df[corr02_col].copy()
sub_test.fillna(sub_test.median(),inplace=True)
x_test = sub_test.values
X = np.vstack((x_train,x_test))
X = preprocessing.scale(X)
x_train = X[0:len(x_train)]
x_test = X[len(x_train):]
model = build_model(x_train,y_train)
subA = model.predict(x_test)
# read submit data
sub_df = pd.read_csv('subA.csv',header=None)
sub_df['Y'] = subA
sub_df.to_csv('github.csv',header=None,index=False)

In [55]:
lin_scores = cross_val_score(model, x_train, y_train,scoring="neg_mean_squared_error", cv=10)
np.mean(lin_scores)

-0.035890758033725143